In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv
import json

# ------------------ Load and Prepare Data ------------------ #
data = pd.read_csv(r'J:\CancerInst\ImmunoTherapy\Lab_Current\Guillaume Mestrallet\Experiments\brcatcga03012024\MLcorrelations7.csv')
X = data.drop(columns=["OS_MONTHS", "OS_STATUS"])
y = Surv.from_dataframe("OS_STATUS", "OS_MONTHS", data)

# ------------------ Define Model and Param Grid ------------------ #
rf_survival = RandomSurvivalForest(random_state=42)

param_grid = {
    "n_estimators": [10, 100, 200, 500],
    "max_depth": [None, 10, 15, 25],
    "min_samples_split": [2, 5, 10, 15],
    "min_samples_leaf": [1, 2, 5, 10],
}

# ------------------ Step 1: Nested CV for Unbiased Performance ------------------ #
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
c_indices = []

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    print(f"\n--- Outer Fold {fold_idx + 1} ---")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Train on outer train set with default params
    base_model = RandomSurvivalForest(random_state=42)
    base_model.fit(X_train, y_train)

    # Evaluate on outer test fold
    y_pred_test = base_model.predict(X_test)
    c_index_test = concordance_index_censored(
        y_test["OS_STATUS"], y_test["OS_MONTHS"], y_pred_test
    )[0]

    print(f"Test C-Index: {c_index_test:.4f}")
    c_indices.append(c_index_test)

print(f"\n✅ Mean Nested CV Test C-Index: {np.mean(c_indices):.4f}")

# ------------------ Step 2: Hyperparameter Tuning on Full Dataset ------------------ #
print("\n🔍 Performing grid search on the full dataset for hyperparameter tuning...")

inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)

grid_search = GridSearchCV(
    rf_survival,
    param_grid=param_grid,
    cv=inner_cv,
    n_jobs=-1,
    scoring=None,  # sksurv uses model.score, which is concordance
    refit=True
)

grid_search.fit(X, y)
best_params = grid_search.best_params_
print("\n🏆 Best Hyperparameters on Full Data:")
print(best_params)

# ------------------ Step 3: Final Model Training ------------------ #
final_model = RandomSurvivalForest(**best_params, random_state=42)
final_model.fit(X, y)

# ------------------ Save Results ------------------ #
results = {
    "fold_c_indices_nested_cv": c_indices,
    "mean_nested_cv_c_index": np.mean(c_indices),
    "selected_params_full_data": best_params,
}

with open("nestedcv_then_final_gridsearch_results.json", "w") as f:
    json.dump(results, f, indent=4)

# ------------------ Optional: Predict on External Cohort ------------------ #
# external_data = pd.read_csv("external_cohort.csv")
# X_ext = external_data.drop(columns=["OS_MONTHS", "OS_STATUS"])
# y_ext = Surv.from_dataframe("OS_STATUS", "OS_MONTHS", external_data)
# y_pred_ext = final_model.predict(X_ext)
# c_index_ext = concordance_index_censored(
#     y_ext["OS_STATUS"], y_ext["OS_MONTHS"], y_pred_ext
# )[0]
# print(f"\nExternal Cohort C-Index: {c_index_ext:.4f}")



--- Outer Fold 1 ---
Test C-Index: 0.7637

--- Outer Fold 2 ---
Test C-Index: 0.7590

--- Outer Fold 3 ---
Test C-Index: 0.8144

--- Outer Fold 4 ---
Test C-Index: 0.6908

--- Outer Fold 5 ---
Test C-Index: 0.7063

✅ Mean Nested CV Test C-Index: 0.7468

🔍 Performing grid search on the full dataset for hyperparameter tuning...

🏆 Best Hyperparameters on Full Data:
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 10}


In [20]:
best_model=final_model
best_model

RandomSurvivalForest(max_depth=10, min_samples_leaf=1, min_samples_split=15,
                     n_estimators=10, random_state=42)

In [21]:
import shap

In [ ]:
explainer2 = shap.Explainer(best_model.predict, X)  # Approximate SHAP for predictions
shap_values2 = explainer2(X, max_evals=6500)
shap.summary_plot(shap_values2, X)


PermutationExplainer explainer:   4%|▍         | 34/802 [12:50<4:52:57, 22.89s/it]

In [ ]:
X_test_sorted = X_test.sort_values(by=['Cyclophosphamide','rna_ESRRG','rna_IRF2','rna_MAP2K6','cg04515986','cg06531741','cg17016000'])
X_test_sel = pd.concat((X_test_sorted.head(3), X_test_sorted.tail(3)))

X_test_sel

pd.Series(best_model.predict(X_test_sel))

surv = best_model.predict_survival_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(best_model.unique_times_, s, where="post", label=str(i))
plt.ylabel("Survival probability")
plt.xlabel("Time in days")
plt.legend()
plt.grid(True)

In [ ]:
X_test_sorted = X_test.sort_values(by=['Cyclophosphamide','rna_ESRRG','rna_IRF2','rna_MAP2K6','cg04515986','cg06531741','cg17016000'])
X_test_sel = pd.concat((X_test_sorted.head(3), X_test_sorted.tail(3)))

X_test_sel

pd.Series(best_model.predict(X_test_sel))

surv = best_model.predict_cumulative_hazard_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(best_model.unique_times_, s, where="post", label=str(i))
plt.ylabel("Cumulative hazard")
plt.xlabel("Time in days")
plt.legend()
plt.grid(True)

In [ ]:
X_test_sorted = X_test.sort_values(by=['AGE','NFKB1|NF-kB-p65_pS536','rna_MAFA','rna_PELO','cg03889226','cg04632671','cg09645888','cg10970251'])
X_test_sel = pd.concat((X_test_sorted.head(3), X_test_sorted.tail(3)))

X_test_sel

pd.Series(best_model.predict(X_test_sel))

surv = best_model.predict_survival_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(best_model.unique_times_, s, where="post", label=str(i))
plt.ylabel("Survival probability")
plt.xlabel("Time in days")
plt.legend()
plt.grid(True)

In [ ]:
X_test_sorted = X_test.sort_values(by=['AGE','NFKB1|NF-kB-p65_pS536','rna_MAFA','rna_PELO','cg03889226','cg04632671','cg09645888','cg10970251'])
X_test_sel = pd.concat((X_test_sorted.head(3), X_test_sorted.tail(3)))

X_test_sel

pd.Series(best_model.predict(X_test_sel))

surv = best_model.predict_cumulative_hazard_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(best_model.unique_times_, s, where="post", label=str(i))
plt.ylabel("Cumulative hazard")
plt.xlabel("Time in days")
plt.legend()
plt.grid(True)

In [ ]:
import pandas as pd
import numpy as np
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored

# === Load new dataset ===
new_data = pd.read_csv(r'J:\CancerInst\ImmunoTherapy\Lab_Current\Guillaume Mestrallet\Experiments\brcatcga03012024\ovarianML.csv')  # <-- Change this path

# === Prepare features (X_new) and survival labels (y_new) ===
# Ensure the column names match the original training dataset structure
X_new = new_data.drop(columns=["OS_MONTHS", "OS_STATUS"])
y_new = Surv.from_dataframe("OS_STATUS", "OS_MONTHS", new_data)

# === Predict risk scores using the trained model ===
y_pred_new = best_model.predict(X_new)

# === Evaluate using Concordance Index ===
# Convert event indicator to boolean
c_index_new = concordance_index_censored(
    y_new['OS_STATUS'].astype(bool),
    y_new['OS_MONTHS'],
    y_pred_new
)

print(f"New Dataset C-Index: {c_index_new[0]:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored

# === Load new dataset ===
new_data = pd.read_csv(r'J:\CancerInst\ImmunoTherapy\Lab_Current\Guillaume Mestrallet\Experiments\brcatcga03012024\uterineML.csv')  # <-- Change this path

# === Prepare features (X_new) and survival labels (y_new) ===
# Ensure the column names match the original training dataset structure
X_new = new_data.drop(columns=["OS_MONTHS", "OS_STATUS"])
y_new = Surv.from_dataframe("OS_STATUS", "OS_MONTHS", new_data)

# === Predict risk scores using the trained model ===
y_pred_new = best_model.predict(X_new)

# === Evaluate using Concordance Index ===
# Convert event indicator to boolean
c_index_new = concordance_index_censored(
    y_new['OS_STATUS'].astype(bool),
    y_new['OS_MONTHS'],
    y_pred_new
)

print(f"New Dataset C-Index: {c_index_new[0]:.4f}")

In [ ]:
import pandas as pd
import numpy as np
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored

# === Load new dataset ===
new_data = pd.read_csv(r'J:\CancerInst\ImmunoTherapy\Lab_Current\Guillaume Mestrallet\Experiments\brcatcga03012024\gliomaML.csv')  # <-- Change this path

# === Prepare features (X_new) and survival labels (y_new) ===
# Ensure the column names match the original training dataset structure
X_new = new_data.drop(columns=["OS_MONTHS", "OS_STATUS"])
y_new = Surv.from_dataframe("OS_STATUS", "OS_MONTHS", new_data)

# === Predict risk scores using the trained model ===
y_pred_new = best_model.predict(X_new)

# === Evaluate using Concordance Index ===
# Convert event indicator to boolean
c_index_new = concordance_index_censored(
    y_new['OS_STATUS'].astype(bool),
    y_new['OS_MONTHS'],
    y_pred_new
)

print(f"New Dataset C-Index: {c_index_new[0]:.4f}")

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc
import numpy as np

min_time = y_test["OS_MONTHS"].min()
max_time = y_test["OS_MONTHS"].max()

# Slightly shrink max time to stay strictly within interval
safe_max_time = max_time * 0.9999

# Generate safe time points
eval_times = np.linspace(min_time, safe_max_time, 10)

# Predict survival functions
pred_surv = best_model.predict_survival_function(X_test)

# Evaluate survival probabilities at the eval_times
pred_surv_matrix = np.asarray([[fn(t) for t in eval_times] for fn in pred_surv])
risk_scores = 1 - pred_surv_matrix

# Compute cumulative/dynamic AUC
cph_auc, mean_auc = cumulative_dynamic_auc(y_train, y_test, risk_scores, eval_times)

#plot
plt.plot(eval_times, cph_auc, marker='o', color='orange', label="UEC BRCA test")
plt.xlabel("Time (days)")
plt.ylabel("Time-dependent AUC")
plt.title("Time-dependent ROC AUC for RSF")
plt.ylim(0, 1)
plt.grid(True)
plt.show()

In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator
import matplotlib.pyplot as plt

# Get event indicator and durations from test set
event = y_test["OS_STATUS"]
time = y_test["OS_MONTHS"]

# Compute KM estimate
km_time, km_survival = kaplan_meier_estimator(event, time)

# Predict survival functions for all test samples
pred_surv = best_model.predict_survival_function(X_test)

# Interpolate all survival functions to a common time grid
# Choose a common grid based on the KM time or a linspace
common_time_grid = np.linspace(0, time.max(), 100)

# Interpolate predicted survival values at these times
interp_surv = np.asarray([[fn(t) for t in common_time_grid] for fn in pred_surv])

# Compute mean predicted survival across all test samples
mean_pred_surv = interp_surv.mean(axis=0)

#plot
plt.step(km_time, km_survival, where="post", label="Kaplan–Meier (observed)")
plt.plot(common_time_grid, mean_pred_surv, label="RSF predicted (mean)", color="orange")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.title("Observed vs Predicted Survival Curves")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
from sksurv.metrics import cumulative_dynamic_auc
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Determine valid evaluation time range from new data
min_time_new = y_new["OS_MONTHS"].min()
max_time_new = y_new["OS_MONTHS"].max()
safe_max_time = max_time_new * 0.9999  # to avoid upper bound violation

# Step 2: Choose time points for AUC evaluation
eval_times_new = np.linspace(min_time_new, safe_max_time, 10)

# Step 3: Predict survival for new data using trained model
pred_surv_new = best_model.predict_survival_function(X_new)
pred_surv_matrix_new = np.asarray([[fn(t) for t in eval_times_new] for fn in pred_surv_new])

# Step 4: Convert to risk scores
risk_scores_new = 1 - pred_surv_matrix_new

# Step 5: Compute cumulative dynamic AUC using training set as reference
cph_auc_new, mean_auc_new = cumulative_dynamic_auc(y_train, y_new, risk_scores_new, eval_times_new)

# Step 6: Plot
plt.plot(eval_times_new, cph_auc_new, marker='o', color='orange', label="UEC AUC")
plt.xlabel("Time (days)")
plt.ylabel("Time-dependent AUC")
plt.ylim(0, 1)
plt.title("Time-dependent ROC AUC on New Data")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator
import matplotlib.pyplot as plt

# Get event indicator and durations from test set
event_new = y_new["OS_STATUS"]
time_new = y_new["OS_MONTHS"]

# Compute KM estimate
km_time_new, km_survival_new = kaplan_meier_estimator(event_new, time_new)

# Predict survival functions for all test samples
pred_surv_new = best_model.predict_survival_function(X_new)

# Interpolate all survival functions to a common time grid
# Choose a common grid based on the KM time or a linspace
common_time_grid_new = np.linspace(0, time_new.max(), 100)

# Interpolate predicted survival values at these times
interp_surv_new = np.asarray([[fn(t) for t in common_time_grid_new] for fn in pred_surv_new])

# Compute mean predicted survival across all test samples
mean_pred_surv_new = interp_surv_new.mean(axis=0)

#plot
plt.step(km_time_new, km_survival_new, where="post", label="Kaplan–Meier (observed)")
plt.plot(common_time_grid_new, mean_pred_surv_new, label="RSF predicted (mean)", color="orange")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.title("Observed vs Predicted Survival Curves")
plt.grid(True)
plt.legend()
plt.show()